# Title

MRI Lesion Segmentation in Multiple Sclerosis Database

# Description

Multiple sclerosis (MS) is the most common inflammatory demyelinating disease of the central nervous system. The disease can appear in multiple types, depending on the duration of the relapse and remitting stages. The initial point of the disease is Clinically Isolated Syndrome (CIS), that is up to 24 hours long. During the following progress, the demyelated regions (so-called lesions) are starting to appear in the brain white matter, causing interconnections between neurons to degrade. 

Multiple tests are used to diagnose MS precisely. Lesion detection and traction through Magnetic Resonanse Imaging (MRI) appears to be pretty sensitive and safe among these methods. MRI is an effective technique to measure MS because MRI uses magnetic fields and radio waves to measure the relative water content in tissues - both normal tissue and abnormal - in the body. Since the layer of myelin the protects the nerve cell is fatty, it repels water. In the areas where myelin has been damaged by MS, the fat is stripped away. With the fat gone, the area holds more water, which shows in MRI either as a bright spot or darkened areas depending on the type of scan that’s used.

The described dataset can be useful to estimate the future progress of the disease since the CIS stage based on MRI scans.

# Experiment

- 38 subjects with diagnosed CIS MS and 20 healthy subjects with different age, gender and degree of disability participated in the experiment.
- MRI was made since 0 month and 6-12 months after the CIS
- EDSS coefficients were estimated for each subject, and were reestimated for MS patients during the second procedure
- Lesions on MR images were segmented manually

### MRI device

Serial longitudinal transversed MR scans was made using a T2-weighted turbo spinecho pulse sequence (repetition time = 4408 ms, echo time = 100 ms, echo spacing = 10.8 ms). The reconstructed image had a slice thickness of 5 mm and a field of view of 230 mm with a pixel resolution of 2.226 pixels/mm. 

### Subjects

Thirty-eight patients (17 males, and 21 females), aged 34.1 ± 10.5 (mean age ± standard deviation), with a CIS of MS and MRI-detectable brain lesions were scanned twice at 1.5 T with an interval of 6-12 months. Additionally, the brains of 20 healthy, age-matched (mean ± SD: 30.8 ± 7.6) volunteers (8 males, and 12 females) were MRI-scanned to allow segmentation and analysis of normal brain white matter 

### Target
The target variable is EDSS score (Expanded Disability Status Scale) in a range from 0 to 10, providing a way of quantifying disability in MS and monitoring the changes over time in level of disability. 
- 0.5 unit increments
- EDSS steps 1.0 to 4.5 refer to people with MS who are able to walk without any aid
- EDSS steps 5.0 to 9.5 are defined by the impairment to walking

Here we are specifically interested in EDSS score > 2 as a cut-off point because above this point neurological signs demonstrate the onset of accumulating disability.
- EDSS(2) -> 20% Stabilized
- EDSS(5) -> 50% Stabilized

### File content
- 1887 MR images in TIFF or BMP format
- 1777 segmented lesions in .plq format. Each file contains lesion boundaries, cropped lesion image, related binary mask
- The data about healthy subjects and gender data are missing

# Hypotheses

1. Shape Features

    1. Maximum length of x and y coords, 
    1. Area, 
    1. perimeter, 
    1. major and minor axis length, 
    1. eccentricity, 
    1. convex area 
    1. volume
    
These features tend to describe the shape and size of lesion. Overall an increment in these directly leads to number of demyelinated axons leads to an increased signal transmission impairment, i.e. patient EDSS score can increase.

2. Texture features

    1. Median Value - intuitively shows the brightness of each ROI. MS Lesions are brighter than NWM.
    1. Contrast - which is a measure of local variation between pixel intensities. MS regions have higher contrast values than NWM regions
    1. Inverse different moment (IDM) – This is related to lesions homogeneity. In NWM, IDM had higher values than in MS regions, implying that NWM were smoother and more homogeneous.
    1. Entropy and Sum of average  - MS lesions attained lower values than NWM ROIs, indicating that the degree of randomness of pixel intensities or textural roughness in MS regions was lower.
    
(TODO provide causal relations with MS disease)
    
3. Spatial features

(TODO provide set of features)

The location of the lesion can be connected with the EDSS score, because it is important what parts of brain were damaged by the inflammation. 

# Questions
- What is Normal Appearing White Matter?

# Related articles

1. C.P. Loizou, V. Murray, M.S. Pattichis, I. Seimenis, M. Pantziaris, C.S. Pattichis, Multi-scale amplitude modulation-frequency modulation (AM-FM) texture analysis of multiple sclerosis in brain MRI images, IEEE Trans. Inform. Tech. Biomed., vol. 15, no. 1, pp. 119-129, 2011.   

1. C.P. Loizou, E.C. Kyriacou, I. Seimenis, M. Pantziaris, S. Petroudi, M. Karaolis, C.S. Pattichis, Brain white matter lesion classification in multiple sclerosis subjects for the prognosis of future disability, Intelligent Decision Technologies Journal (IDT), vol. 7, pp. 3-10, 2013.

1. C.P. Loizou, M. Pantziaris, C.S. Pattichis, I. Seimenis, Brain MRI Image normalization in texture analysis of multiple sclerosis, J. Biomed. Graph. & Comput., vol. 3, no.1, pp. 20-34, 2013. 

1. C.P. Loizou, S. Petroudi, I. Seimenis, M. Pantziaris, C.S. Pattichis, Quantitative texture analysis of brain 
white matter lesions derived from T2-weighted MR images in MS patients with clinically isolated syndrome, J. Neuroradiol., acepted. 

1. https://www.nationalmssociety.org/Symptoms-Diagnosis/Diagnosing-Tools/MRI